In [1]:
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import os 
%matplotlib inline

In [2]:
df = pd.read_csv("D:\\Learn\\Uni\\ML\\CapstoneProject\\data\\healthcare-dataset-stroke-data.csv")
df = df.drop('id', axis=1)
df['bmi'] = df['bmi'].fillna(df['bmi'].mean())

Pre Processing

In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['gender'] = le.fit_transform(df['gender'])
df['ever_married'] = le.fit_transform(df['ever_married'])
df['work_type'] = le.fit_transform(df['work_type'])
df['Residence_type'] = le.fit_transform(df['Residence_type'])
df['smoking_status'] = le.fit_transform(df['smoking_status'])

In [4]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
features = ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status']

In [5]:
X = df[features]
y = df['stroke']

In [6]:
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Creating the Neural Networ

In [8]:
import tensorflow
from tensorflow.keras import layers, models
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

In [9]:
inputs = keras.Input(shape=(10,))
a = layers.Dense(64, activation='relu')(inputs)
x = layers.Dropout(0.6)(a)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.6)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.6)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.6)(x)
x = layers.concatenate([a, x])
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.6)(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dropout(0.6)(x)
output = layers.Dense(1, activation='sigmoid')(x)

nn = keras.Model(inputs, output, name="stroke_predictor")

In [10]:
nn.compile(optimizer=tensorflow.optimizers.Adam(learning_rate=0.0001),
            loss='binary_crossentropy', metrics=['accuracy'])
early_stopper = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

#Trains the neural network
history = nn.fit(X_train, y_train, epochs=1000, callbacks=[early_stopper], validation_data=(X_test, y_test), verbose=0)

Test

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, recall_score ,precision_score, f1_score

In [12]:
test_preds = nn.predict(X_test)
threshold = 0.25
test_preds = [1. if i > threshold else 0. for i in test_preds]
test_acc = accuracy_score(y_test, test_preds)
test_rec = recall_score(y_test, test_preds)
test_pre = precision_score(y_test, test_preds)
print('Test Set Metrics')
print('Model Accuracy:', test_acc)
print('Model Recall:', test_rec)
print('Model Precision:', test_pre)

61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Test Set Metrics
Model Accuracy: 0.5018013381369018
Model Recall: 1.0
Model Precision: 0.5012879958784132
